In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pyttsx3

# STEP 2: Initialize Text-to-Speech Engine
engine = pyttsx3.init()

def speak(text):
    engine.say(text)
    engine.runAndWait()

# STEP 3: Define the Agent Function
def Agent(data_path):
    # Load Data
    data = pd.read_csv(data_path)
    
    # Data Preprocessing
    columns_to_use = [
        "Sleep Quality (1-10)",
        "Physical Activity (hrs/week)",
        "Stress Level (1-10)",
        "Mood Score (1-10)",
        "Diagnosis"
    ]
    data_subset = data[columns_to_use].copy()

    # Label encoding target variable (Diagnosis)
    label_encoder = LabelEncoder()
    data_subset["Diagnosis_Encoded"] = label_encoder.fit_transform(data_subset["Diagnosis"])

    # Define features (X) and target (y)
    X = data_subset[["Sleep Quality (1-10)", "Physical Activity (hrs/week)", "Stress Level (1-10)", "Mood Score (1-10)"]]
    y = data_subset["Diagnosis_Encoded"]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a Decision Tree Classifier
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)

    # Evaluate the model
    y_pred = clf.predict(X_test)
    print("Classification Report:\n")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

    # Function to get user input through typing
#     def get_user_input_typing():
#         while True:
#             try:
#                 print("\nAnswer the following questions (1-10):")
#                 sleep_quality = int(input("How would you rate your sleep quality? "))
#                 physical_activity = int(input("How many hours of physical activity do you have per week? "))
#                 stress_level = int(input("How would you rate your stress level? "))
#                 mood_score = int(input("How would you rate your overall mood? "))
                
#                 if all(1 <= x <= 10 for x in [sleep_quality, stress_level, mood_score]) and physical_activity >= 0:
#                     return [[sleep_quality, physical_activity, stress_level, mood_score]]
#                 else:
#                     print("Please enter valid ratings (1-10) and non-negative physical activity.")
#             except ValueError:
#                 print("Invalid input. Please enter integer values.")

    # Function to ask questions and get user input by typing
    def ask_questions_and_get_input():
        speak("Welcome to the Mental Health Monitoring Assistant!")
        print("Welcome to the Mental Health Monitoring Assistant!")

        # List of questions
        questions = [
            "How would you rate your sleep quality on a scale of 1 to 10?",
            "How many hours of physical activity do you do per week?",
            "How would you rate your stress level on a scale of 1 to 10?",
            "How would you rate your overall mood on a scale of 1 to 10?"
        ]
        
        # Ask each question and get response
        responses = []
        for question in questions:
            speak(question)
            print(question)  # Print the question for reference
            response = input("Your answer: ").strip()
            try:
                # Convert response to integer
                response = int(response)
                if 1 <= response <= 10 or (question == questions[1] and response >= 0):
                    responses.append(response)
                else:
                    speak(f"Please provide a valid answer for {question}. Please try again")
                    print(f"Please provide a valid answer for {question}. Please try again")
                    return None
            except ValueError:
                speak("Invalid input. Please enter a number. Try again")
                print("Invalid input. Please enter a number. Try again")
                return None
        
        return [responses]

    # Get User Input
    user_input = ask_questions_and_get_input()
    if user_input is None:
        return

    # Predict Diagnosis
    prediction = clf.predict(user_input)
    diagnosis = label_encoder.inverse_transform(prediction)

    # Provide Diagnosis and Suggestions
    speak(f"Based on your responses, the predicted diagnosis is: {diagnosis[0]}")
    print(f"\nBased on your responses, the predicted diagnosis is: {diagnosis[0]}")

    
    suggestions = {
        "Major Depressive Disorder ": "Engage in regular physical activity, practice mindfulness, and consider reaching out to a therapist.",
        "Generalized Anxiety": "Try deep breathing exercises, reduce caffeine intake, and explore relaxation techniques like yoga.",
        "Bipolar Disorder": "Maintain a routine, monitor your mood, and consult a mental health professional for medication management.",
        "Panic Disorder": "Practice grounding techniques, engage in deep breathing exercises, and seek professional support if needed."
    }

    if diagnosis[0] in suggestions:
        speak(f"Here are some suggestions for you: {suggestions[diagnosis[0]]}")
        print(f"Suggestions: {suggestions[diagnosis[0]]}")
        
        

# Run the Agent Function
Agent("/Users/hamza/Documents/mental_health_diagnosis_treatment_.csv")